In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import sparse
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Load the count matrix
# Assuming it's a csv file with rows as genes and columns as cells
counts = pd.read_csv("counts.csv", index_col=0)

# Transpose so that rows are cells and columns are genes
counts = counts.transpose()

# Normalization (CPM normalization)
counts = counts.div(counts.sum(axis=1), axis=0)
counts = counts.mul(10**6)

# Logarithmize the data
counts = np.log1p(counts)

# Scale data to zero mean and unit variance
scaler = StandardScaler()
counts_scaled = pd.DataFrame(scaler.fit_transform(counts), columns=counts.columns, index=counts.index)

# Run PCA
pca = PCA(n_components=50)
pca_result = pca.fit_transform(counts_scaled)

# Plot explained variance
plt.plot(range(pca.n_components_), np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.show()


In [1]:
#clustering
from sklearn.cluster import KMeans

# Assume we have 10 clusters
n_clusters = 10

# Run k-means clustering on the PCA result
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(pca_result)

# Assign clusters back to original dataframe
counts['cluster'] = kmeans.labels_


NameError: name 'pca_result' is not defined

In [ ]:
#DEG
import statsmodels.api as sm
from scipy import stats

# Perform a t-test for each gene to find differentially expressed genes
def differential_expression(data, labels):
    cluster_labels = np.unique(labels)
    results = []
    for gene in data.columns:
        group1 = data.loc[labels == cluster_labels[0], gene]
        group2 = data.loc[labels == cluster_labels[1], gene]
        t_stat, p_val = stats.ttest_ind(group1, group2)
        results.append((gene, t_stat, p_val))
    # Correct for multiple testing
    _, p_vals_corrected, _, _ = sm.stats.multipletests([x[2] for x in results], method='fdr_bh')
    results_corrected = [(x[0], x[1], p) for x, p in zip(results, p_vals_corrected)]
    return results_corrected

differential_expression_results = differential_expression(counts, kmeans.labels_)



In [ ]:
#Gene Set Enrichment Anlasis
import gseapy as gp

# Let's assume that `differential_expression_results` is the list of tuples you got from the differential expression analysis
# and each tuple contains a gene name, t-statistic, and corrected p-value

# Convert the differential expression results into a format that gseapy can use
gene_list = sorted(differential_expression_results, key=lambda x: x[2])
gene_list = [x[0] for x in gene_list]

# Run Enrichr
# 'gene_sets' can be the name of a gene set library in Enrichr
enrichr_results = gp.enrichr(gene_list=gene_list, description='pathway', gene_sets='KEGG_2016', outdir='enrichr_output')

# Display the results
print(enrichr_results.results)


In [ ]:
#Heatmap display
import seaborn as sns
import matplotlib.pyplot as plt

# Let's assume that `differential_expression_results` is the list of tuples you got from the differential expression analysis
# and each tuple contains a gene name, t-statistic, and corrected p-value

# Select DEGs based on some threshold
degs = [x[0] for x in differential_expression_results if x[2] < 0.05]

# Subset the original dataframe to include only DEGs
deg_df = counts[degs]

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(deg_df, cmap="RdBu_r")
plt.show()
